# Places Analysis
In this notebook we'll do some analysis of the Google "PlaceVisit" data using pandas. We'll plot our results using Matplotlib.

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
# Set this to the name of your places file
PLACES_PATH = 'places.csv'
# TODO: allow setting start and end date
places = pd.read_csv(PLACES_PATH, sep='|', encoding='utf-8', parse_dates=['start_timestamp', 'end_timestamp'])
print(f'Data has {places.shape[0]} rows and {places.shape[1]} columns')
places.info()

In [ ]:
print('Top ten places, by number of records')
places['name'].value_counts()[:10]

In [ ]:
# Calculate time spent per place.
# Note: this may provide seemingly strange results. 
# Personal addresses (where you live) likely don't have a "name", and
# therefore won't show up in the results. To see them, group by 'address'
print('Top ten places, by duration:')
places['duration'] = places['end_timestamp'] - places['start_timestamp']
time_spent = places.groupby('name')['duration'].sum()
time_spent.sort_values(inplace=True, ascending=False)
time_spent[:10]

In [ ]:
# TODO: countries